In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from itertools import product
from competition.titanic.FeaturesEngineering import *
import warnings
warnings.filterwarnings("error", category=FutureWarning)

from sklearn.pipeline import Pipeline
pd.set_option("display.max_columns", None)

In [3]:
df = pd.read_csv("../data/train.csv", delimiter=",")

features = [x for x in df.columns if x != "Survived"]
x, y = df[features], df["Survived"].T
print(x.shape)
print(y.shape)

(891, 11)
(891,)


In [4]:
X_train, X_cv, y_train, y_cv = train_test_split(x, y, train_size=0.8, random_state=1)

In [4]:
# Verify the final features
feature_engineer = engineer_features()
X_transformed = feature_engineer.fit_transform(X_train)
names = feature_engineer.get_feature_names_out()
final_df = pd.DataFrame(X_transformed, columns=names)
print(final_df.shape)
final_df.head()

(712, 70)


,HasCabin,A,B,C,D,E,F,G,T,U,Name_Capt,Name_Col,Name_Don,Name_Dr,Name_Lady,Name_Major,Name_Master,Name_Miss,Name_Mlle,Name_Mme,Name_Mr,Name_Mrs,Name_Rev,Name_the Countess,AgeAndSex_adult_f,AgeAndSex_adult_m,AgeAndSex_child_f,AgeAndSex_child_m,AgeAndSex_nan_f,AgeAndSex_nan_m,AgeAndSex_senior_f,AgeAndSex_senior_m,AgeAndSex_teenager_f,AgeAndSex_teenager_m,AgeAndTitle_adult_Col,AgeAndTitle_adult_Don,AgeAndTitle_adult_Dr,AgeAndTitle_adult_Lady,AgeAndTitle_adult_Major,AgeAndTitle_adult_Miss,AgeAndTitle_adult_Mlle,AgeAndTitle_adult_Mme,AgeAndTitle_adult_Mr,AgeAndTitle_adult_Mrs,AgeAndTitle_adult_Rev,AgeAndTitle_adult_the Countess,AgeAndTitle_child_Master,AgeAndTitle_child_Miss,AgeAndTitle_child_Mr,AgeAndTitle_nan_Master,AgeAndTitle_nan_Miss,AgeAndTitle_nan_Mr,AgeAndTitle_nan_Mrs,AgeAndTitle_senior_Capt,AgeAndTitle_senior_Col,AgeAndTitle_senior_Miss,AgeAndTitle_senior_Mr,AgeAndTitle_senior_Mrs,AgeAndTitle_teenager_Miss,AgeAndTitle_teenager_Mr,AgeAndTitle_teenager_Mrs,FamilySize,IsAlone,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,PassengerId,Pclass,Fare
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,302.0,3.0,23.2500
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,310.0,1.0,56.9292
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,517.0,2.0,10.5000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,121.0,2.0,73.5000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,571.0,2.0,10.5000


In [5]:
# Evaluate hypeparameters one by one
# max_depth_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
# min_samples_split_list = range(30, 90, 10)
# n_estimators_list = range(20, 60, 10)
# train_acc = []
# cv_acc = []
# for min_samples in [1]:
#     model = Pipeline(steps=[
#         ("preprocessor", engineer_features()),
#         ("classifier", RandomForestClassifier(n_estimators=30,
#                                               max_depth=9,
#                                               min_samples_split=50,
#                                               n_jobs=-1,
#                                               bootstrap=True,
#                                               random_state=1))
#     ])
#     model.fit(X_train, y_train)
#     y_train_hat = model.predict(X_train)
#     train_accuracy = np.mean(y_train_hat == y_train)
#     y_cv_hat = model.predict(X_cv)
#     cv_accuracy = np.mean(y_cv == y_cv_hat)
#     train_acc.append(train_accuracy)
#     cv_acc.append(cv_accuracy)
#     print(
#         f"Max depth: {min_samples},Train accuracy: {train_accuracy}, CV accuracy: {cv_accuracy}")

# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.plot(min_samples_list, train_acc)
# plt.plot(min_samples_list, cv_acc)
# plt.xlabel("min_child_weight_list")
# plt.legend(["Train accuracy", "CV accuracy"])
# plt.show()

In [6]:
max_depth_list = [6, 7, 8, 9]
min_samples_split_list = [30, 40, 50, 60]
n_estimators_list = [40, 50, 60, 150]
# max_depth_list = [2, 3, 4, 5, 6, 7, 8]
# min_samples_split_list = [30, 50, 100, 150, 200, 250, 300]
# n_estimators_list = [50, 100, 150, 200]
def eval_models(X_train, y_train, X_cv, y_cv, max_depth, min_samples_split, n_estimators, feature_transformer):
    model = Pipeline(steps=[
        ("preprocessor", feature_transformer),
        ("classifier", RandomForestClassifier(n_estimators=n_estimators,
                                              max_depth=max_depth,
                                              min_samples_split=min_samples_split,
                                              n_jobs=-1,
                                              bootstrap = True,
                                              random_state=1))
    ])
    model.fit(X_train, y_train)
    y_train_hat = model.predict(X_train)
    train_accuracy = np.mean(y_train_hat == y_train)
    y_cv_hat = model.predict(X_cv)
    cv_accuracy = np.mean(y_cv == y_cv_hat)
    print(
        f"Max depth: {max_depth}, Min sample split: {min_samples_split}, N estimators: {n_estimators}, Train accuracy: {train_accuracy}, CV accuracy: {cv_accuracy}")
    return model, max_depth, min_samples_split, n_estimators, train_accuracy, cv_accuracy


model_evals = [
    eval_models(X_train, y_train, X_cv, y_cv, max_depth, min_samples_split, n_estimators, engineer_features()) for
    max_depth, min_samples_split, n_estimators in product(max_depth_list, min_samples_split_list, n_estimators_list)]

best_model_params = max(model_evals, key=lambda x: x[5])
print(best_model_params[-5:])

Max depth: 6, Min sample split: 30, N estimators: 40, Train accuracy: 0.8553370786516854, CV accuracy: 0.7877094972067039
Max depth: 6, Min sample split: 30, N estimators: 50, Train accuracy: 0.8539325842696629, CV accuracy: 0.7877094972067039
Max depth: 6, Min sample split: 30, N estimators: 60, Train accuracy: 0.8595505617977528, CV accuracy: 0.7821229050279329
Max depth: 6, Min sample split: 30, N estimators: 150, Train accuracy: 0.8581460674157303, CV accuracy: 0.7932960893854749
Max depth: 6, Min sample split: 40, N estimators: 40, Train accuracy: 0.8455056179775281, CV accuracy: 0.8156424581005587
Max depth: 6, Min sample split: 40, N estimators: 50, Train accuracy: 0.851123595505618, CV accuracy: 0.7932960893854749
Max depth: 6, Min sample split: 40, N estimators: 60, Train accuracy: 0.8469101123595506, CV accuracy: 0.8100558659217877
Max depth: 6, Min sample split: 40, N estimators: 150, Train accuracy: 0.851123595505618, CV accuracy: 0.7988826815642458
Max depth: 6, Min sample

In [8]:
# max_depth_list = [6, 7, 8, 9]
# min_samples_split_list = [30, 40, 50, 60]
# n_estimators_list = [40, 50, 60, 150]
max_depth_list = [2, 3, 4, 5, 6, 7, 8]
min_samples_split_list = [30, 50, 100, 150, 200, 250, 300]
n_estimators_list = [50, 100, 150, 200]

model = Pipeline(steps=[
    ("preprocessor", engineer_features()),
    ("classifier", RandomForestClassifier(n_estimators=40,
                                          bootstrap=True,
                                          random_state=1))
])
param_grid = {
    "classifier__n_estimators": n_estimators_list,
    "classifier__max_depth": max_depth_list,
    "classifier__min_samples_split": min_samples_split_list,
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring="accuracy",
    cv=cv,
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

grid.fit(X_train, y_train)

print(f"Grid best score: {grid.best_score_}")
print(f"Grid best param: {grid.best_params_}")

Fitting 5 folds for each of 196 candidates, totalling 980 fits


/Users/lamp/Documents/Project/perso/machine-learning/kaggle-comp/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['T'] will be ignored
  warnings.warn(
/Users/lamp/Documents/Project/perso/machine-learning/kaggle-comp/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['T'] will be ignored
  warnings.warn(
/Users/lamp/Documents/Project/perso/machine-learning/kaggle-comp/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['T'] will be ignored
  warnings.warn(
/Users/lamp/Documents/Project/perso/machine-learning/kaggle-comp/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['T'] will be ignored
  warnings.warn(
/Users/lamp/Documents/Project/perso/machine-learning/kaggle-comp/.venv/lib/python3.13/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['T'

Grid best score: 0.8455234905939131
Grid best param: {'classifier__max_depth': 7, 'classifier__min_samples_split': 30, 'classifier__n_estimators': 100}


In [9]:
df_test = pd.read_csv("../data/test.csv", delimiter=",")

best_model = grid.best_estimator_
y_test_hat = best_model.predict(df_test)
output = pd.DataFrame({
    "PassengerId": df_test["PassengerId"],
    "Survived": y_test_hat
})
output.to_csv("submission.csv", index=False)